Idea: what if we run a lot more optimizations but with a much shorter time limit. Can we bootstrap the warmstarts to make that short horizon count?

In [1]:
using Revise

In [2]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using Gurobi
using Flux
using ProgressMeter
using MLDataPattern
using JLD2
using Plots; gr()

Plots.GRBackend()

In [3]:
import LCPSim
import LearningMPC
import BoxValkyries
reload("LearningMPC")
reload("BoxValkyries")

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/BoxValkyries.ji for module BoxValkyries.


In [4]:
robot = BoxValkyries.BoxValkyrie(true, LCPSim.planar_revolute_base);
mvis = MechanismVisualizer(robot)
IJuliaCell(mvis)

Listening on 127.0.0.1:7001...
zmq_url=tcp://127.0.0.1:6001
web_url=http://127.0.0.1:7001/static/


MeshCat.IJuliaCell(MeshCat.ViewerWindow(ZMQ.Context(Ptr{Void} @0x000000000708e530, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x0000000007067760, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000006fc37c0, 56, (1, 0), Condition(Any[]), (false, false), 0))]), ZMQ.Socket(Ptr{Void} @0x0000000007067760, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000006fc37c0, 56, (1, 0), Condition(Any[]), (false, false), 0)), "http://127.0.0.1:7001/static/", "tcp://127.0.0.1:6001", MeshCat.ZMQServer.ZMQWebSocketBridge(ip"127.0.0.1", Set{WebSockets.WebSocket}(), MeshCat.SceneTrees.SceneNode(Nullable{Array{UInt8,1}}(), Nullable{Array{UInt8,1}}(), Dict("meshcat"=>MeshCat.SceneTrees.SceneNode(#NULL, #NULL, Dict("robot"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x72, 0x6f, 0x62, 0x6f, 0x74], Dict("world"=>MeshCat.SceneTrees.SceneNode(#NULL, #NULL, Dict("dummy"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x64, 0x75, 0x6d, 0x6d, 0x79], Dict("base"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x64, 0x75, 0x6d, 0x6d, 0x79, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict("base_revolute"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x5f, 0x72, 0x65, 0x76, 0x6f, 0x6c, 0x75, 0x74, 0x65], Dict("core"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x6c, 0x75, 0x74, 0x65, 0x2f, 0x63, 0x6f, 0x72, 0x65], Dict("rh_dummy"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x2f, 0x72, 0x68, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79], Dict("after_core_to_rh_rotation"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x5f, 0x72, 0x6f, 0x74, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict("geometry_2"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"rh"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x2f, 0x72, 0x68], Dict("after_core_to_rh_extension"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x65, 0x78, 0x74, 0x65, 0x6e, 0x73, 0x69, 0x6f, 0x6e], Dict("geometry_3"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String,MeshCat.SceneTrees.SceneNode}()))))))),"rf_dummy"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x2f, 0x72, 0x66, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79], Dict("after_core_to_rf_rotation"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x5f, 0x72, 0x6f, 0x74, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict("geometry_6"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x36], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x36], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"rf"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0

In [19]:
function create_net()
    net = Chain(
        Dense(22, 64, elu),
        Dense(64, 64, elu),
        Dense(64, 11)
    )
    loss = (x, y) -> Flux.mse(net(x), y)
    net, loss
end

create_net (generic function with 1 method)

In [22]:
net, loss = create_net()
net_controller = x -> Flux.Tracker.data(net(state_vector(x)))
net_params = params(net)
optimizer = Flux.Optimise.ADADelta(net_params)

xstar = BoxValkyries.nominal_state(robot)

mpc_params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=10,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, 
        TimeLimit=5, 
        MIPGap=1e-1, 
        MIPGapAbs=5e-1,
        FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

Q, R = BoxValkyries.default_costs(robot)
feet = findbody.(robot.mechanism, ["lf", "rf"])
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, mpc_params.Δt, Point3D.(default_frame.(feet), 0., 0., 0.))
lqrsol.S .= 1 ./ mpc_params.Δt .* Q

mpc_controller = LearningMPC.MPCController(robot.mechanism, 
    robot.environment, mpc_params, lqrsol, 
    [net_controller, lqrsol]);

sample_sink = LearningMPC.MPCSampleSink{Float64}(true)
playback_sink = LearningMPC.PlaybackSink(mvis, mpc_params.Δt)

mpc_controller.callback = LearningMPC.call_each(
    sample_sink,
    playback_sink,
)

live_viewer = LearningMPC.live_viewer(mvis)


dagger_controller = LearningMPC.call_each(
    LearningMPC.dagger_controller(
        mpc_controller,
        net_controller,
        0.2),
    live_viewer
    )

termination = x -> begin
    (configuration(x)[2] < 0.5 || 
     configuration(x)[3] > π/4 ||
     configuration(x)[3] < -π/4)
end

dataset = LearningMPC.Dataset(lqrsol)

function collect_into!(data::Vector{<:LearningMPC.Sample})
    empty!(sample_sink)
    LearningMPC.randomize!(x0, xstar, 0.0, 1.5)
    results = LCPSim.simulate(x0, 
        dagger_controller,
        robot.environment, mpc_params.Δt, 30, 
        mpc_params.lcp_solver;
        termination=termination);
    append!(data, sample_sink.samples)
end

x0 = MechanismState{Float64}(robot.mechanism)

features(s::LearningMPC.Sample) = (s.state, s.uJ[:, 1])

Academic license - for non-commercial use only
Academic license - for non-commercial use only


features (generic function with 1 method)

In [24]:
datasets = Vector{LearningMPC.Dataset{Float64}}()
all_training_data = Vector{Tuple{Vector{Float64}, Vector{Float64}}}()
all_validation_data = Vector{Tuple{Vector{Float64}, Vector{Float64}}}()
losses = Vector{Tuple{Float64, Float64}}()

@showprogress for i in 1:20
    dataset = LearningMPC.Dataset(lqrsol)
    for i in 1:2
        collect_into!(dataset.training_data)
    end
    collect_into!(dataset.testing_data)
    collect_into!(dataset.validation_data)
    append!(all_training_data, features.(dataset.training_data))
    append!(all_validation_data, features.(dataset.validation_data))
    
    @time for i in 1:10
        Flux.train!(loss, shuffleobs(all_training_data), optimizer)
        push!(losses, 
            (mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_training_data),
             mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_validation_data)))
    end
    push!(datasets, dataset)
    
    jldopen("box-val-revolute.jld2", "w") do file
        file["datasets"] = datasets
        file["net"] = net
        file["lqrsol"] = lqrsol
        file["mpc_params"] = Dict(
            "Δt" => mpc_params.Δt,
            "horizon" => mpc_params.horizon,
        )
        file["all_training_data"] = all_training_data
        file["all_validation_data"] = all_validation_data
        file["losses"] = losses
    end
    
    plt = plot(first.(losses), label="training")
    plot!(plt, last.(losses), label="validation")
    ylims!(plt, (0, ylims(plt)[2]))
    display(plt)
end

captured: ErrorException("Unrecognized solution status: loaded")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 
 
 
 
 
 training 
 
 
 
 validation

  0.128445 seconds (58.96 k allocations: 5.459 MiB)


Progress:   5%|██                                       |  ETA: 0:29:19

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 
 
 
 
 
 training 
 
 
 
 validation

...

  0.686763 seconds (1.01 M allocations: 111.837 MiB, 5.32% gc time)


Progress:  95%|███████████████████████████████████████  |  ETA: 0:02:15

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
<polyline clip-path="url(#clip3702)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.1292,121.384 31.899,128.738 34.6688,135.618 37.4385,142.019 40.2083,148.128 42.9781,153.918 45.7479,159.455 48.5176,164.824 51.2874,169.962 54.0572,174.953 
 56.827,151.76 59.5968,159.975 62.3665,167.591 65.1363,174.718 67.9061,181.362 70.6759,187.722 73.4456,193.663 76.2154,199.301 78.9852,204.662 81.755,209.626 
 84.5248,114.004 87.2945,119.502 90.0643,124.555 92.8341,129.193 95.6039,133.549 98.3736,137.627 101.143,141.574 103.913,145.311 106.683,148.853 109.453,152.395 
 112.223,134.607 114.992,140.482 117.762,145.862 120.532,150.859 123.302,155.349 126.071,159.371 128.841,163.216 131.611,166.894 134.381,170.219 137.151,173.351 
 139.92,149.876 142.69,154.307 145.46,158.386 148.23,162.132 150.999,165.257 153.769,168.31 156.539,171.068 159.309,173.676 162.079,176.176 164.848,178.427 
 167.618,163.298 170.388,167.195 173.158,170.711 175.927,174.103 178.697,177.219 181.467,180.273 184.237,183.07 187.007,185.875 189.776,188.476 192.546,190.976 
 195.316,195.509 198.086,197.646 200.855,199.745 203.625,201.746 206.395,203.567 209.165,205.311 211.935,206.932 214.704,208.372 217.474,209.886 220.244,211.292 
 223.014,188.9 225.783,191.671 228.553,193.949 231.323,195.931 234.093,197.916 236.863,199.666 239.632,201.258 242.402,202.767 245.172,204.265 247.942,205.536 
 250.711,211.416 253.481,212.918 256.251,214.217 259.021,215.637 261.791,216.92 264.56,218.066 267.33,219.249 270.1,220.218 272.87,221.205 275.639,222.142 
 278.409,216.357 281.179,217.546 283.949,218.613 286.719,219.619 289.488,220.501 292.258,221.338 295.028,222.17 297.798,222.876 300.567,223.556 303.337,224.279 
 306.107,215.193 308.877,216.087 311.647,216.917 314.416,217.761 317.186,218.542 319.956,219.35 322.726,220.071 325.495,220.875 328.265,221.623 331.035,222.35 
 333.805,213.104 336.575,214.1 339.344,214.993 342.114,215.816 344.884,216.625 347.654,217.345 350.423,217.953 353.193,218.627 355.963,219.299 358.733,220.003 
 361.503,224.297 364.272,224.991 367.042,225.637 369.812,226.163 372.582,226.81 375.351,227.327 378.121,227.833 380.891,228.431 383.661,228.862 386.431,229.516 
 389.2,216.406 391.97,217.115 394.74,217.891 397.51,218.646 400.279,219.599 403.049,220.209 405.819,220.835 408.589,221.45 411.359,221.995 414.128,222.601 
 416.898,219.238 419.668,220.044 422.438,220.876 425.207,221.288 427.977,221.844 430.747,222.54 433.517,223.142 436.287,223.679 439.056,224.25 441.826,224.851 
 444.596,214.369 447.366,215.253 450.135,215.699 452.905,216.44 455.675,217.025 458.445,217.582 461.215,218.152 463.984,218.692 466.754,219.237 469.524,219.793 
 472.294,222.634 475.063,223.515 477.833,224.15 480.603,224.763 483.373,225.369 486.143,225.968 488.912,226.415 491.682,226.921 494.452,227.39 497.222,227.904 
 499.991,225.599 502.761,226.113 505.531,226.918 508.301,227.449 511.071,227.951 513.84,228.621 516.61,229.158 519.38,229.657 522.15,230.106 524.919,230.55 
 527.689,229.664 530.459,230.236 533.229,230.502 535.999,231.091 538.768,231.63 541.538,232.156 544.308,232.663 547.078,233.087 549.847,233.432 552.617,233.979 
 555.387,232.859 558.157,233.419 560.927,233.879 563.696,234.428 566.466,234.835 569.236,235.28 572.006,235.546 574.775,236.111 577.545,236.349 580.315,236.64 
 
 "/>
<polyline clip-path="url(#clip3702)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.1292,118.871 31.899,122.338 34.6688,125.491 37.4385,128.345 40.2083,131.006 42.9781,133.414 45.7479,135.664 48.5176,137.772 51.2874,139.71 54.0572,141.543 
 56.827,11.811 59.5968,12.7606 62.3665,13.5532 65.1363,14.2644 67.9061,14.7361 70.6759,15.102 73.4456,15.3693 76.2154,15.4715 78.9852,15.3016 81.755,15.0608 
 84.5248,110.209 87.2945,114.3 90.0643,118.04 92.8341,121.504 95.

  0.705001 seconds (1.06 M allocations: 116.384 MiB, 5.08% gc time)


Progress: 100%|█████████████████████████████████████████| Time: 0:44:22


In [33]:
x0 = MechanismState{Float64}(robot.mechanism)
LearningMPC.randomize!(x0, xstar, 0.0, 1.0)
results = LCPSim.simulate(x0, 
    net_controller,
    robot.environment, mpc_params.Δt, 100, 
    mpc_params.lcp_solver,
    termination=x -> false);
LearningMPC.playback(mvis, results, 0.05)

In [35]:
online_mpc_controller = LearningMPC.OnlineMPCController(
    robot.mechanism,
    robot.environment,
    mpc_params,
    lqrsol,
    [lqrsol, net_controller]
);

LearningMPC.OnlineMPCController{Float64,LearningMPC.MPCParams{Gurobi.GurobiSolver,Gurobi.GurobiSolver},RigidBodyDynamics.MechanismState{Float64,Float64,Float64,TypeSortedCollections.TypeSortedCollection{Tuple{Array{RigidBodyDynamics.Joint{Float64,RigidBodyDynamics.Prismatic{Float64}},1},Array{RigidBodyDynamics.Joint{Float64,RigidBodyDynamics.Revolute{Float64}},1},Array{RigidBodyDynamics.Joint{Float64,RigidBodyDynamics.Fixed{Float64}},1}},3},TypeSortedCollections.TypeSortedCollection{Tuple{Array{RigidBodyDynamics.Spatial.GeometricJacobian{StaticArrays.SArray{Tuple{3,1},Float64,2,3}},1},Array{RigidBodyDynamics.Spatial.GeometricJacobian{StaticArrays.SArray{Tuple{3,1},Float64,2,3}},1},Array{RigidBodyDynamics.Spatial.GeometricJacobian{StaticArrays.SArray{Tuple{3,0},Float64,2,0}},1}},3},TypeSortedCollections.TypeSortedCollection{Tuple{Array{RigidBodyDynamics.Spatial.WrenchMatrix{StaticArrays.SArray{Tuple{3,5},Float64,2,15}},1},Array{RigidBodyDynamics.Spatial.WrenchMatrix{StaticArrays.SArray{

In [41]:
x0 = MechanismState{Float64}(robot.mechanism)
LearningMPC.randomize!(x0, xstar, 0.0, 1.0)
results = LCPSim.simulate(x0, 
    online_mpc_controller,
    robot.environment, mpc_params.Δt, 100, 
    mpc_params.lcp_solver,
    termination=x -> false);
LearningMPC.playback(mvis, results, 0.05)

cost.(warmstarts) = [43.3878, 57.9562]
cost.(warmstarts) = [14.3816, 128.244]
cost.(warmstarts) = [9.47058, 83.5072]
cost.(warmstarts) = [4.76008, 91.9679]
cost.(warmstarts) = [2.59061, 63.6374]
cost.(warmstarts) = [1.61861, 56.9274]
cost.(warmstarts) = [1.03664, 46.9526]
cost.(warmstarts) = [0.771254, 44.6018]
cost.(warmstarts) = [0.695533, 42.0804]
cost.(warmstarts) = [0.726613, 38.8269]
cost.(warmstarts) = [0.812213, 37.7745]
cost.(warmstarts) = [0.920329, 36.0123]
cost.(warmstarts) = [1.081, 35.1758]
cost.(warmstarts) = [1.1755, 36.2615]
cost.(warmstarts) = [1.27987, 36.9602]
cost.(warmstarts) = [1.43556, 36.659]
cost.(warmstarts) = [1.49673, 37.1217]
cost.(warmstarts) = [1.55826, 37.0315]
cost.(warmstarts) = [1.69655, 36.8085]
cost.(warmstarts) = [1.72253, 38.034]
cost.(warmstarts) = [1.75582, 36.2201]
cost.(warmstarts) = [1.86907, 36.6439]
cost.(warmstarts) = [1.85505, 36.5424]
cost.(warmstarts) = [1.93842, 35.6882]
cost.(warmstarts) = [1.92259, 35.9033]
cost.(warmstarts) = [2.00

In [40]:
LearningMPC.playback(mvis, results, 0.05)

This really isn't working well. The training isn't very effective (the final loss is relatively high, and there's a lot of overfitting), and the trained policy isn't very effective (LQR seems to outperform it consistently). 

In addition, I'm concerned that the bootstrap idea isn't terribly good, because even if it did work, we would still have a lot of bad old samples in the training set. We *might* be able to decay their importance over time, but that's a whole other kind of hyperparameter that I'm not sure I want to tune.

In addition, the online MPC version seems little better than LQR, as the network's warmstart is never chosen. 

So far, the only case where the network does something non-trivial correctly is the simple hopper, which is simple enough that the MIQP is almost fast enough to run online as-is. 

That's frustrating. 

How can we do better? 

* Learn better
* Use more domain knowledge

I have a feeling that I should re-visit learning the optimal value function instead of the policy. There are a couple of potential advantages:

* It's a much simpler function (scalar instead of 11-dimensional)
* I can exploit my knowledge of the model when I compute `u` from dJ/dx
* It might be more amenable to bootstrapping

To that last point: currently if I terminate an optimization prematurely, I end up with a bad (x, u) pair in the dataset. Even if the network learns a better warmstart near that x, my dataset still contains the bad pair. 

On the other hand, if I'm instead learning (x, (J_lower_bound, J_upper_bound)), then a bad sample just consists of a bound that's too wide. If I later learn a tighter bound for that sample, then the original bad data has no effect! 

